In [ ]:
# !git clone https://github.com/transmuteAI/trailmet.git

In [ ]:
cd trailmet

/content/trailmet


In [ ]:
# !git checkout chipnet-base

In [ ]:
!git pull

remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (5/5), done.
remote: Total 6 (delta 4), reused 4 (delta 4), pack-reused 1
Unpacking objects: 100% (6/6), done.
From https://github.com/transmuteAI/trailmet
   a807d84..6f91d78  chipnet-base -> origin/chipnet-base
Updating a807d84..6f91d78
error: Your local changes to the following files would be overwritten by merge:
	trailmet/algorithms/prune/chipnet.py
Please commit your changes or stash them before you merge.
Aborting


In [ ]:
import torch
import matplotlib.pyplot as plt
from torchvision import transforms
from trailmet.models import ModelsFactory
from trailmet.datasets.classification import DatasetFactory


In [ ]:
import yaml

In [ ]:
root = "/content/trailmet/experiments/chipnet"

In [ ]:
%%writefile experiments/chipnet/resnet50_cifar10.yaml
CHIPNET_ARGS:
    A: 10

PRETRAIN:
    EPOCHS: 1

PRUNE:
    EPOCHS: 1

FINETUNE:
    EPOCHS: 2

Overwriting experiments/chipnet/resnet50_cifar10.yaml


In [ ]:
import os
with open(os.path.join(root, "resnet50_cifar10.yaml"), 'r') as stream:
    data_loaded = yaml.safe_load(stream)

In [ ]:
print(data_loaded)

{'CHIPNET_ARGS': {'A': 10}, 'PRETRAIN': {'EPOCHS': 1}, 'PRUNE': {'EPOCHS': 1}, 'FINETUNE': {'EPOCHS': 2}}


In [ ]:
model = ModelsFactory.create_model('resnet18', 10, False, insize=32)

In [ ]:
pretrained_weights = "https://download.pytorch.org/models/resnet18-f37072fd.pth"

In [ ]:
from torch.hub import load_state_dict_from_url
weights = load_state_dict_from_url(pretrained_weights)

In [ ]:
from trailmet.datasets.classification import DatasetFactory
data_root = "/content/data_dir"

In [ ]:
mkdir /content/data_dir

mkdir: cannot create directory ‘/content/data_dir’: File exists


In [ ]:
train_transform = transforms.Compose(
[transforms.ToTensor()])

val_transform = transforms.Compose(
[transforms.ToTensor()])

test_transform = transforms.Compose(
[transforms.ToTensor()])

transforms1 = {
    'train': train_transform, 
    'val': val_transform, 
    'test': test_transform}
def train_target_transform(label):
    return label

def val_target_transform(label):
    return label

def test_target_transform(label):
    return label

target_transforms = {
    'train': None, 
    'val': None, 
    'test': None}


cifar_dataset = DatasetFactory.create_dataset(name = 'CIFAR10', 
                                        root = data_root,
                                        split_types = ['train', 'val', 'test'],
                                        val_fraction = 0.2,
                                        transform = transforms1,
                                        target_transform = target_transforms
                                        )

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [ ]:
train_loader = torch.utils.data.DataLoader(
        cifar_dataset['train_dataset'], batch_size=64, 
        sampler=cifar_dataset['train_sampler'],
        num_workers=0
    )
val_loader = torch.utils.data.DataLoader(
        cifar_dataset['val_dataset'], batch_size=64, 
        sampler=cifar_dataset['val_sampler'],
        num_workers=0
    )
test_loader = torch.utils.data.DataLoader(
        cifar_dataset['test_dataset'], batch_size=64, 
        sampler=cifar_dataset['test_sampler'],
        num_workers=0
    )

In [ ]:
from trailmet.algorithms.prune.chipnet import ChipNet

In [ ]:
a = ChipNet(model, {'train': train_loader, 'val': val_loader, 'test': test_loader}, **data_loaded)

In [ ]:
a.compress_model()

starting pretraining


100%|██████████| 157/157 [00:03<00:00, 52.13it/s, acc=0.368, loss=1.99]


**Saving model**


100%|██████████| 157/157 [00:03<00:00, 50.79it/s, acc=0.367, loss=1.98]


Test Accuracy: 0.3668 | Valid Accuracy: 0.368
pretraining done
starting pruning
preparing model for pruning
Starting epoch 1 / 1


...

**Saving model**


100%|██████████| 157/157 [00:03<00:00, 39.65it/s, acc=0.729, loss=0.787]


**Saving model**


100%|██████████| 157/157 [00:04<00:00, 36.37it/s, acc=0.728, loss=0.786]

Test Accuracy: 0.7278 | Valid Accuracy: 0.7285
finetuning done


In [ ]:
sum([0,0,True])

1